## 데이터 전처리

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('csv/train.csv')

In [3]:
# 차트 한글폰트
plt.rc('font', family='Malgun Gothic')

In [4]:
drop_cols = ['ID','압축천연가스(CNG)','경유','가솔린','하이브리드','액화석유가스(LPG)', '판매도시','판매구역',]

# '연료' 컬럼 생성 및 값 할당
df['연료'] = 0
# '액화석유가스(LPG)'에 해당하는 경우 4 할당
df.loc[df['액화석유가스(LPG)'] == 1, '연료'] = 0
# '경유'에 해당하는 경우 1 할당
df.loc[df['경유'] == 1, '연료'] = 1
# '가솔린'에 해당하는 경우 2 할당
df.loc[df['가솔린'] == 1, '연료'] = 2
# '압축천연가스(CNG)'에 해당하는 경우 3 할당
df.loc[df['압축천연가스(CNG)'] == 1, '연료'] = 3
# '하이브리드'에 해당하는 경우 4 할당
df.loc[df['하이브리드'] == 1, '연료'] = 4

# 기존 컬럼 제거
df = df.drop(columns=drop_cols)
df = df[df['주행거리'] != df['주행거리'].max()]
df = df[df["주행거리"] >= 200]
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56698 entries, 0 to 56697
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   생산년도    56698 non-null  int64  
 1   모델출시년도  56698 non-null  int64  
 2   브랜드     56698 non-null  object 
 3   차량모델명   56698 non-null  object 
 4   주행거리    56698 non-null  int64  
 5   배기량     56698 non-null  int64  
 6   가격      56698 non-null  float64
 7   연료      56698 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.5+ MB


In [5]:
# 레이블 인코딩(Label Encoding) - 차량모델명 컬럼
label_encoder = LabelEncoder()
df['차량모델명'] = label_encoder.fit_transform(df['차량모델명'])
# df['판매도시'] = label_encoder.fit_transform(df['판매도시'])

# 원핫 인코딩(One-Hot Encoding) - 차량모델명 컬럼
df = pd.get_dummies(df)

## 스케일링

In [6]:
# # 로그 스케일링
# df['주행거리'] = np.log(df['주행거리'])

In [7]:
# # 스탠다드 스케일링
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(df)
# scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

In [8]:
y_car_df = df['가격']
X_car_df = df.drop('가격', axis=1)

In [9]:
X_train, X_test, y_train, y_test= train_test_split(X_car_df, y_car_df, test_size=0.2)

In [10]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
linear_pred = linear_model.predict(X_test)
print(f'''
      정확도 : {r2_score(y_test, linear_pred)}
      RMSE : {np.sqrt(mean_squared_error(y_test, linear_pred))}
      MAE : {mean_absolute_error(y_test, linear_pred)}
      ''')


      정확도 : 0.7765270193881385
      RMSE : 16.251387896687742
      MAE : 11.952479335417612
      


## 피처 확인

In [11]:
df_feature_importances = pd.DataFrame({'Features':X_car_df.columns,'계수':linear_model.coef_})
df_feature_importances

,Features,계수
0,생산년도,4.123215
1,모델출시년도,1.234517
2,차량모델명,0.104468
3,주행거리,-0.000095
4,배기량,0.021968
5,연료,0.964275
6,브랜드_audi,19.320667
7,브랜드_bmw,5.869069
8,브랜드_citroen,-6.159071
9,브랜드_fiat,-15.403952


In [12]:
print('절편 값 : ' ,linear_model.intercept_)

절편 값 :  -10752.721395842802


In [13]:
# 회기 계수가 큰 값 순으로 정렬
coeff = pd.Series(data=np.round(linear_model.coef_, 1), index=X_car_df.columns)
coeff.sort_values(ascending=False)

브랜드_audi             19.3
브랜드_mercedes-benz    13.5
브랜드_honda             8.7
브랜드_bmw               5.9
브랜드_volkswagen        5.4
브랜드_peugeot           4.7
생산년도                  4.1
브랜드_volvo             3.7
모델출시년도                1.2
연료                    1.0
브랜드_mazda             0.6
브랜드_nissan            0.3
차량모델명                 0.1
브랜드_toyota            0.1
주행거리                 -0.0
배기량                   0.0
브랜드_skoda             0.0
브랜드_seat             -2.8
브랜드_kia              -3.3
브랜드_ford             -4.7
브랜드_mitsubishi       -5.0
브랜드_citroen          -6.2
브랜드_opel             -6.9
브랜드_hyundai          -8.5
브랜드_renault          -9.3
브랜드_fiat            -15.4
dtype: float64

In [14]:
# cross_val_score()로 5 fold 세트로 MSE를 구한 뒤 이를 기반으로 다시 RMSE를 구함.
neg_mse_scores = cross_val_score(linear_model, X_car_df, y_car_df, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

print(f'''
      5 folds의 개별 Negative MSE 스코어 : {np.round(neg_mse_scores, 2)}
      5 folds의 개별 rmse 스코어 : {np.round(rmse_scores,2)}
      5 folds의 평균 rmse {avg_rmse}
      ''')


      5 folds의 개별 Negative MSE 스코어 : [-266.35 -265.74 -267.29 -261.9  -262.64]
      5 folds의 개별 rmse 스코어 : [16.32 16.3  16.35 16.18 16.21]
      5 folds의 평균 rmse 16.271995734660543
      
